# The SDS011 Air Quality Sensor 

http://aqicn.org/sensor/sds011/es/

Developed by Marcelo Rovai

- 21 August 2019

Based on: [py-sds011](https://github.com/ikalchev/py-sds011) by [Ivan Kalchev ](https://www.linkedin.com/in/ivan-kalchev-b342b26b/)

In [1]:
from sds011 import *
from datetime import datetime
import time

In [2]:
print(datetime.now())

2019-08-20 21:08:07.053149


In [3]:
def time_now():
    dateTimeObj = datetime.now()
    timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
    return timestampStr

In [5]:
sensor = SDS011("/dev/ttyUSB0")

In [6]:
# Turn-on sensor
sensor.sleep(sleep=False)

In [7]:
# Turn-off sensor
sensor.sleep(sleep=True)

In [8]:
ts = time.gmtime()
print(time.strftime("%Y-%m-%d %H:%M:%S", ts))

2019-08-21 01:08:34


In [8]:
# Converting datetime object to string
dateTimeObj = datetime.now()
timestamp = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
print('Current Timestamp : ', timestamp)

Current Timestamp :  20-Aug-2019 (19:51:02)


In [18]:
sensor.sleep(sleep=False)
time.sleep(10)
pmt_2_5, pmt_10 = sensor.query()
print (time_now(), end='')
print(f"    PMT2.5: {pmt_2_5} μg/m3    ", end='')
print(f"PMT10 : {pmt_10} μg/m3")
sensor.sleep(sleep=True)
time.sleep(2)

20-Aug-2019 (20:00:19)    PMT2.5: 2.4 μg/m3    PMT10 : 3.9 μg/m3


---

In [19]:
from gpiozero import CPUTemperature

In [20]:
cpu = CPUTemperature()
print(f"CPU Temp: {cpu.temperature:.3}oC")

CPU Temp: 62.3oC


---
## AQI Converter
A library to convert between AQI value and pollutant concentration (µg/m³ or ppm) using the following algorithms:
- United States Environmental Protection Agency (EPA)
- China Ministry of Environmental Protection (MEP)

https://pypi.org/project/python-aqi/

In [12]:
import aqi

In [23]:
aqi_2_5 = aqi.to_iaqi(aqi.POLLUTANT_PM25, str(pmt_2_5))
aqi_10 = aqi.to_iaqi(aqi.POLLUTANT_PM10, str(pmt_10))

In [24]:
print (time_now(), end='')
print(f"    AQI (PMT2.5): {aqi_2_5}    ", end='')
print(f"AQI(PMT10) : {aqi_10}")

20-Aug-2019 (20:01:38)    AQI (PMT2.5): 10    AQI(PMT10) : 3


---

In [9]:
def get_data(n=3):
        sensor.sleep(sleep=False)
        pmt_2_5 = 0
        pmt_10 = 0
        time.sleep(10)
        for i in range (n):
            x = sensor.query()
            pmt_2_5 = pmt_2_5 + x[0]
            pmt_10 = pmt_10 + x[1]
            time.sleep(2)
        pmt_2_5 = round(pmt_2_5/n, 1)
        pmt_10 = round(pmt_10/n, 1)
        sensor.sleep(sleep=True)
        time.sleep(2)
        return pmt_2_5, pmt_10

In [30]:
for i in range (5):
    print (i, get_data())

0 (6.5, 14.6)
1 (6.4, 17.0)
2 (6.4, 17.9)
3 (6.5, 17.8)
4 (6.6, 17.1)


In [10]:
def conv_aqi(pmt_2_5, pmt_10):
    aqi_2_5 = aqi.to_iaqi(aqi.POLLUTANT_PM25, str(pmt_2_5))
    aqi_10 = aqi.to_iaqi(aqi.POLLUTANT_PM10, str(pmt_10))
    return aqi_2_5, aqi_10

In [13]:
pmt_2_5, pmt_10 = get_data()
aqi_2_5, aqi_10 = conv_aqi(pmt_2_5, pmt_10)
print (time_now(), end='')
print(f"   PMT2.5: {pmt_2_5}μg/m3 (AQI:{aqi_2_5})  ", end='')
print(f"PMT10: {pmt_10}μg/m3 (AQI:{aqi_10})")

20-Aug-2019 (21:09:49)   PMT2.5: 5.2μg/m3 (AQI:22)  PMT10: 9.2μg/m3 (AQI:8)


---

## Logging Data

In [76]:
pwd

'/home/pi/Documents/Air_Sensor'

In [ ]:
with open("/home/pi/Documents/Air_Sensor/air_quality.csv", "a") as log:
    while True:
        pmt_2_5, pmt_10 = get_data()
        aqi_2_5, aqi_10 = conv_aqi(pmt_2_5, pmt_10)
        dt = datetime.now()
        log.write("{},{},{},{},{}\n".format(dt, pmt_2_5, aqi_2_5, pmt_10, aqi_10))
        time.sleep(30)

In [85]:
def save_log():        
    with open("/home/pi/Documents/Air_Sensor/air_quality.csv", "a") as log:
        pmt_2_5, pmt_10 = get_data()
        aqi_2_5, aqi_10 = conv_aqi(pmt_2_5, pmt_10)
        dt = datetime.now()
        log.write("{},{},{},{},{}\n".format(dt, pmt_2_5, aqi_2_5, pmt_10, aqi_10))
    log.close()   

In [86]:
save_log()
time.sleep(30)
save_log()
time.sleep(30)
save_log()

---

## Sending Data to ThingSpeak

In [ ]:
# Need to install the Paho client library.  
# sudo pip install paho-mqtt

In [16]:
import paho.mqtt.publish as publish
import psutil

ThingSpeak credentials and inicialization

In [17]:
channelID = "YOUR CHANNEL ID"
apiKey = "YOUR WRITE KEY"
topic = "channels/" + channelID + "/publish/" + apiKey
mqttHost = "mqtt.thingspeak.com"

MQTT Protocol Initialization

In [18]:
# Conventional TCP socket on port 1883.  
# This connection method is the simplest and requires the least system resources.
tTransport = "tcp"
tPort = 1883
tTLS = None

Defining Payload

In [35]:
tPayload = "field1=" + str(pmt_2_5)+ "&field2=" + str(aqi_2_5)+ "&field3=" + str(pmt_10)+ "&field4=" + str(aqi_10)

Testing sending data one time

In [36]:
pmt_2_5, pmt_10 = get_data()
aqi_2_5, aqi_10 = conv_aqi(pmt_2_5, pmt_10)

In [37]:
print ("[INFO] Data prepared to be uploaded")
try:
    publish.single(topic, payload=tPayload, hostname=mqttHost, port=tPort, tls=tTLS, transport=tTransport)
    print ("[INFO] Data sent for 4 fields: ", pmt_2_5, aqi_2_5, pmt_10, aqi_10)
except:
    print ("[INFO] Failure in sending data")

[INFO] Data prepared to be uploaded
[INFO] Data sent for 4 fields:  6.9 29 17.7 16


Sending data continuosly

In [14]:
def save_log():        
    with open("/YOUR PATH HERE/air_quality.csv", "a") as log:
        dt = datetime.now()
        log.write("{},{},{},{},{}\n".format(dt, pmt_2_5, aqi_2_5, pmt_10, aqi_10))
    log.close()  

In [ ]:
# Sending all data to ThingSpeak every 1 minute
while(True): 
        pmt_2_5, pmt_10 = get_data()
        aqi_2_5, aqi_10 = conv_aqi(pmt_2_5, pmt_10)
        tPayload = "field1=" + str(pmt_2_5)+ "&field2=" + str(aqi_2_5)+ "&field3=" + str(pmt_10)+ "&field4=" + str(aqi_10)
        try:
            publish.single(topic, payload=tPayload, hostname=mqttHost, port=tPort, tls=tTLS, transport=tTransport)
            save_log()
        except (KeyboardInterrupt):
            break
        except:
            print ("[INFO] Failure in sending data") 
        time.sleep(60)